<a href="https://colab.research.google.com/github/texaspandaa/NZ/blob/main/topicmodelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
%matplotlib inline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**load data**

In [6]:
drive.mount('drive', True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at drive


In [7]:
data = pd.read_csv('drive/My Drive/NZ/fackcheck headline data/headlines.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
data.head(6)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,date,statement,rating,fact_check,harm_PC1,veracity
0,1,"December 14, 2022",Grant Wahl died because of the COVID-19 vaccine.,false,politifact,-0.525641,False
1,2,"December 14, 2022","Sen. Kyrsten Sinema ""voted with the president ...",mostly-true,politifact,-1.126343,True
2,3,"December 14, 2022","“For the last several months, wages have gone ...",mostly-true,politifact,-1.146951,True
3,4,"December 14, 2022",The Federal Reserve “has suspended the availab...,false,politifact,-1.052610,False
4,7,"December 13, 2022","Large machinery is “required to move 500,000 p...",false,politifact,-1.110309,False
5,8,"December 13, 2022",The war in Ukraine is “all over.”,false,politifact,-0.405668,False


In [9]:
data_statement = data['statement']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


 **Tokenization**

In [10]:
data_statement_p = data_statement.map(lambda x:gensim.utils.simple_preprocess(x, deacc=True))
# print(type(data_statement_p))
# data_statement_p.head(3)
data_statement_p = list(data_statement_p)
print(data_statement_p[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['grant', 'wahl', 'died', 'because', 'of', 'the', 'covid', 'vaccine']


**bigrams and trigrams**

In [11]:
#bulding bigram and trigram models
bigram = gensim.models.Phrases(data_statement_p, min_count=3)
trigram = gensim.models.Phrases(bigram[data_statement_p], threshold=100)

#get sentence clubbed as bigram trigrams
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# print(trigram_mod[bigram_mod[data_statement_p[1]]])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**remove stop words then build trigrams**

In [12]:
stop_words = stopwords.words('english')
data_statement_s = [ [word for word in text if word not in stop_words] for text in data_statement_p]

#form trigrams
data_statement_tri = [ trigram_mod[bigram_mod[text]] for text in data_statement_s]

#print(data_statement_tri[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**lemma**

In [13]:
!python3 -m spaCy download en

/usr/bin/python3: No module named spaCy


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
nlp = spacy.load('en_core_web_sm')
data_statement_l = []
#data_statement_temp = [[nlp(' '.join(text) )] for text in data_statement_tri]
for text in data_statement_tri:
  doc = nlp(' '.join(text))
  data_statement_l.append([token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])

print(data_statement_l)

#aaa = [' '.join(text) for text in data_statement_tri]
#print(aaa)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['grant', 'wahl', 'die', 'covid_vaccine'], ['vote', 'president', 'time'], ['last', 'several', 'month', 'wage', 'go', 'price', 'go'], ['federal_reserve', 'suspend', 'availability', 'new', 'currency'], ['large', 'machinery', 'require', 'move', 'pound', 'earth', 'order', 'get', 'mineral', 'need', 'single', 'tesla', 'car', 'battery'], ['war', 'ukraine'], ['video_show', 'chuck', 'grassley', 'demand', 'joe_biden', 'resignation'], ['early', 'vote', 'count', 'poll', 'close', 'evidence'], ['recent', 'blood_clot', 'stroke', 'major', 'athlete', 'unwanted', 'effect', 'covid_vaccine'], ['human_meat', 'project', 'use', 'donate', 'body', 'organ', 'create', 'alternative', 'meat', 'consumption', 'option'], ['jag', 'charge', 'treason', 'seditious', 'conspiracy'], ['albert_bourla', 'resign', 'say', 'mrna_vaccine', 'safe'], ['sign', 'election', 'certification'], ['elon_musk', 'tweet', 'democrat', 'pay', 'ceo', 'twitter', 'million', 'dollar', 'block', 'delete', 'people', 'page', 'election'], ['covid', 'ja

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**create dictionary and corpus**

In [17]:
#create dictionary
id2word = corpora.Dictionary(data_statement_l)

#create corpus
texts = data_statement_l

#term doc freq
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:3])
print(id2word[2])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1)], [(7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]
grant


**LDA**

In [52]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 30,
                                            random_state= 11,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
for topic in lda_model.print_topics():
  print(topic)

print(len(lda_model.print_topics()))

doc_lda = lda_model[corpus]
print(doc_lda)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(4, '0.000*"metal" + 0.000*"dewalt" + 0.000*"nebula" + 0.000*"january" + 0.000*"unrestricted" + 0.000*"adida" + 0.000*"sportswear" + 0.000*"hover" + 0.000*"large_storm" + 0.000*"official_twitter"')
(18, '0.204*"discover" + 0.027*"camp" + 0.000*"sportswear" + 0.000*"unrestricted" + 0.000*"large_storm" + 0.000*"hover" + 0.000*"metal" + 0.000*"dewalt" + 0.000*"adida" + 0.000*"january"')
(25, '0.268*"allow" + 0.080*"seek" + 0.002*"headline" + 0.000*"quot" + 0.000*"official_twitter" + 0.000*"metal" + 0.000*"dewalt" + 0.000*"large_storm" + 0.000*"mower" + 0.000*"hover"')
(5, '0.117*"admit" + 0.093*"business" + 0.056*"view" + 0.051*"together" + 0.000*"quot" + 0.000*"photograph_show" + 0.000*"large_storm" + 0.000*"hover" + 0.000*"official_twitter" + 0.000*"adida"')
(10, '0.208*"time" + 0.070*"compare" + 0.020*"likely" + 0.000*"quot" + 0.000*"adida" + 0.000*"large_storm" + 0.000*"official_twitter" + 0.000*"sportswear" + 0.000*"hover" + 0.000*"exposition"')
(16, '0.167*"video" + 0.114*"increase"

**viz**

In [53]:
pyLDAvis.enable_notebook()
visualization = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
visualization

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.084107  0.033364       1        1  10.308109
17    -0.152794  0.138446       2        1   7.652184
22    -0.296426 -0.297544       3        1   7.384215
9     -0.172683  0.217462       4        1   6.660485
29    -0.131986  0.087898       5        1   5.579980
20    -0.064735  0.020910       6        1   5.495767
15    -0.006028 -0.002447       7        1   5.127704
8     -0.027558  0.004327       8        1   4.556893
27     0.006439 -0.005595       9        1   4.080731
11     0.000584 -0.004156      10        1   4.027998
2     -0.044380  0.010989      11        1   3.899481
21     0.022365 -0.008818      12        1   3.498157
26    -0.023741  0.003082      13        1   3.271547
13     0.051992 -0.013282      14        1   2.889441
23     0.012541 -0.006788      15        1   2.883442
14    -0.009477 -0.001219      16        1   2.730539
0      0.009599 -0.006141      17        1   2.608699
24     0.061376 -0.014025      18        1   2.558357
7      0.035448 -0.011081      19        1   2.552964
19     0.064708 -0.014253      20        1   2.384842
1      0.047291 -0.012728      21        1   1.651119
3      0.078111 -0.013716      22        1   1.557034
28     0.078680 -0.013497      23        1   1.546373
12     0.079162 -0.013174      24        1   1.191264
16     0.075240 -0.014261      25        1   1.081758
10     0.078738 -0.013467      26        1   0.923372
5      0.078412 -0.013643      27        1   0.736792
25     0.077369 -0.013929      28        1   0.689349
18     0.079321 -0.012776      29        1   0.332249
4      0.076538 -0.009940      30        1   0.139157, topic_info=        Term         Freq        Total Category  logprob  loglift
72       say  5593.000000  5593.000000  Default  30.0000  30.0000
85    people  1382.000000  1382.000000  Default  29.0000  29.0000
62       use  1487.000000  1487.000000  Default  28.0000  28.0000
313    woman  1666.000000  1666.000000  Default  27.0000  27.0000
414      man  1633.000000  1633.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
25      move     0.012284   134.233577  Topic30  -9.3621  -2.7217
26      need     0.012284   351.566481  Topic30  -9.3621  -3.6845
27     order     0.012284   624.173329  Topic30  -9.3621  -4.2585
28     pound     0.012284    86.368380  Topic30  -9.3621  -2.2807
29   require     0.012284   264.457535  Topic30  -9.3621  -3.3998

[1252 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
228       9  0.937963   ability
331      13  0.993475  abortion
186      12  0.969231  activist
291       9  0.985712  activity
424       9  0.983744       add
...     ...       ...       ...
290      11  0.990927       win
313       3  0.998357     woman
155       2  0.988278    worker
156       3  0.995460      year
449       2  0.989780  year_old

[254 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 18, 23, 10, 30, 21, 16, 9, 28, 12, 3, 22, 27, 14, 24, 15, 1, 25, 8, 20, 2, 4, 29, 13, 17, 11, 6, 26, 19, 5])

In [54]:
pyLDAvis.save_html(visualization, "D:\\NZ project\\data\\LDAModel1.html")
pyLDAvis.save_html(visualization, "drive/My Drive/NZ/LDAModel1.html")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**berttopic**